In [ ]:
#import lib

In [23]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import collections
from sklearn.preprocessing import LabelEncoder
import imblearn
from collections import Counter
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks,ClusterCentroids
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# import data

In [3]:
wine = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',delimiter=";")

In [4]:
#eda

In [5]:
quality = wine["quality"].values
category = []
for num in quality:
    if num<5:
        category.append("Low")
    elif num>6:
        category.append("High")
    else:
        category.append("Midium")
category = pd.DataFrame(data=category, columns=["category"])
data = pd.concat([wine,category],axis=1)
data.drop(columns="quality",axis=1,inplace=True)
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [6]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=2018)

# build model

## random forest

In [17]:
rfc = RandomForestClassifier(n_estimators=250,random_state = 2018,oob_score=True)
rfc.fit(X_train, y_train)
print(rfc.oob_score_)

0.8455844818785094


## SVM

In [18]:
# strtifiedKFold
skf = StratifiedKFold(n_splits=4)

In [20]:
# grid search before resample 
pipe_svm=Pipeline([('clf', svm.SVC())])   
grid_params = dict(clf__C=[0.1, 0.3, 1, 3, 10],
                   clf__gamma=[0.1, 0.3, 1, 3, 10],
                   clf__kernel=['rbf','sigmoid'])  
gs_svm = GridSearchCV(estimator=pipe_svm,  
                  param_grid=grid_params,
                  scoring='accuracy',
                  cv=skf)
gs_svm.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__C': [0.1, 0.3, 1, 3, 10], 'clf__gamma': [0.1, 0.3, 1, 3, 10], 'clf__kernel': ['rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [21]:
print(gs_svm.best_score_)

0.8083205717202654

## Decision Tree

In [25]:
clf = Pipeline([
        ('scl', StandardScaler()),
        ('pca', PCA(random_state=42)),
        ('clf', DecisionTreeClassifier(random_state=42))])  

criterion = ['gini', 'entropy']
splitter = ['best']
max_depth = [8, 9, 10, 11, 15, 20, 25]
min_samples_leaf = [2, 3, 5]
class_weight = ['balanced', None] 

param_grid =\
    [{ 'clf__class_weight': class_weight
      ,'clf__criterion': criterion
      ,'clf__splitter': splitter
      ,'clf__max_depth': max_depth
      ,'clf__min_samples_leaf': min_samples_leaf
}]

gs_dt = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_dt.fit(X_train,y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:    3.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            ma...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'clf__class_weight': ['balanced', None], 'clf__criterion': ['gini', 'entropy'], 'clf__splitter': ['best'], 'clf__max_depth': [8, 9, 10, 11, 15, 20, 25], 'clf__min_samples_leaf': [2, 3, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [27]:
print(gs_dt.best_score_)

0.7595712098009189


## KNN

In [28]:
# knn with pca grid search
pipe_knn_pca = Pipeline([('pca', PCA()), ('clf', KNeighborsClassifier())])
grid_params_knn_pca = dict(pca__n_components=range(1, 11), clf__n_neighbors=range(1, 10))

gs_knn_pca = GridSearchCV(estimator=pipe_knn_pca, param_grid=grid_params_knn_pca, scoring='accuracy')

gs_knn_pca.fit(X_train, y_train)

/Users/tianqiluke/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'pca__n_components': range(1, 11), 'clf__n_neighbors': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [29]:
print(gs_knn_pca.best_score_)

0.7391526288922919

**Random Forest gives best result**

In [33]:
#check its performance on test
pred_rfc = rfc.predict(X_test)
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy on Test data is %s" % accuracy_score(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.79      0.63      0.70       209
           1       0.67      0.11      0.20        35
           2       0.87      0.95      0.91       736

   micro avg       0.85      0.85      0.85       980
   macro avg       0.77      0.56      0.60       980
weighted avg       0.84      0.85      0.84       980

The RF model accuracy on Test data is 0.8510204081632653


In [34]:
# relabel back : 0 means good, 1 for low, 2 for medium for better visualization
y_test_re = list(y_test)
for i in range(len(y_test_re)):
  if y_test_re[i] == 0:
    y_test_re[i] = "good"
  if y_test_re[i] == 1:
    y_test_re[i] = "low"
  if y_test_re[i] == 2:
    y_test_re[i] = "medium"
pred_rfc_re = list(pred_rfc)
for i in range(len(pred_rfc_re)):
  if pred_rfc_re[i] == 0:
    pred_rfc_re[i] = "good"
  if pred_rfc_re[i] == 1:
    pred_rfc_re[i] = "low"
  if pred_rfc_re[i] == 2:
    pred_rfc_re[i] = "medium"
y_actu = pd.Series(y_test_re, name='Actual')
y_pred = pd.Series(pred_rfc_re, name='Predicted')
rfc_confusion = pd.crosstab(y_actu, y_pred)

In [35]:
rfc_confusion

Predicted,good,low,medium
Actual,,,
good,131,0,78
low,0,4,31
medium,35,2,699


# Resampling 

In [56]:
################### resample ##################
# under sample "2" ； over sample 1,0
smt = ClusterCentroids(ratio = {2:1500})
X_sm,y_sm = smt.fit_sample(X_train,y_train)
smt2 =SMOTE(ratio = {0:1500,1:1500})
X_sm2,y_sm2 = smt2.fit_sample(X_sm,y_sm)

## Random Forest

In [81]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
labelencoder_y2 = LabelEncoder()
y_train_sm2 = labelencoder_y2.fit_transform(y_sm2)
sc_X2 = StandardScaler()
X_sm22 = sc_X2.fit_transform(X_sm2)
X_test2 = sc_X2.transform(X_test)

In [82]:
rfc_rs = RandomForestClassifier(n_estimators=150,random_state = 2018,oob_score=True)
# might need CV for parameter tuning here
rfc_rs.fit(X_sm22, y_train_sm2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=True, random_state=2018, verbose=0, warm_start=False)

In [83]:
rfc_rs.oob_score_

0.8613333333333333

In [84]:
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy is %s" % accuracy_score(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       209
           1       0.00      0.00      0.00        35
           2       0.75      1.00      0.86       736

   micro avg       0.75      0.75      0.75       980
   macro avg       0.25      0.33      0.29       980
weighted avg       0.56      0.75      0.64       980

The RF model accuracy is 0.7510204081632653


/Users/tianqiluke/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## SVM_rs

In [71]:
# grid search after resample 
pipe_svm=Pipeline([('clf', svm.SVC())])   
grid_params = dict(clf__C=[0.1, 0.3, 1, 3, 10],
                   clf__gamma=[0.1, 0.3, 1, 3, 10],
                   clf__kernel=['rbf','sigmoid'])  
gs_svm_sm = GridSearchCV(estimator=pipe_svm,  
                  param_grid=grid_params,
                  scoring='accuracy',
                  cv=skf)
gs_svm_sm.fit(X_sm2, y_sm2)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__C': [0.1, 0.3, 1, 3, 10], 'clf__gamma': [0.1, 0.3, 1, 3, 10], 'clf__kernel': ['rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [65]:
gs_svm_sm.best_score_

0.8264444444444444

## DT_rs

In [66]:
clf = Pipeline([
        ('scl', StandardScaler()),
        ('pca', PCA(random_state=42)),
        ('clf', DecisionTreeClassifier(random_state=42))])  

criterion = ['gini', 'entropy']
splitter = ['best']
max_depth = [8, 9, 10, 11, 15, 20, 25]
min_samples_leaf = [2, 3, 5]
class_weight = ['balanced', None] 

param_grid =\
    [{ 'clf__class_weight': class_weight
      ,'clf__criterion': criterion
      ,'clf__splitter': splitter
      ,'clf__max_depth': max_depth
      ,'clf__min_samples_leaf': min_samples_leaf
}]

gs_dt_rs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_dt_rs.fit(X_sm2,y_sm2)

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 318 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:    7.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            ma...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'clf__class_weight': ['balanced', None], 'clf__criterion': ['gini', 'entropy'], 'clf__splitter': ['best'], 'clf__max_depth': [8, 9, 10, 11, 15, 20, 25], 'clf__min_samples_leaf': [2, 3, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [67]:
gs_dt_rs.best_score_

0.7475555555555555

## KNN_rs

In [68]:
# knn with pca grid search
pipe_knn_pca = Pipeline([('pca', PCA()), ('clf', KNeighborsClassifier())])
grid_params_knn_pca = dict(pca__n_components=range(1, 11), clf__n_neighbors=range(1, 10))

gs_knn_pca_rs = GridSearchCV(estimator=pipe_knn_pca, param_grid=grid_params_knn_pca, scoring='accuracy')

gs_knn_pca_rs.fit(X_sm2, y_sm2)

/Users/tianqiluke/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'pca__n_components': range(1, 11), 'clf__n_neighbors': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [69]:
gs_knn_pca_rs.best_score_

0.772

**SVM gives best result**(check RF)

In [73]:
# model after resample
svm_rs_predicted = gs_svm_sm.predict(X_test)
print(classification_report(y_test, svm_rs_predicted))
print("---------------------------------------------------------")
print("The accuracy score of SVM (resampled) is %s" % accuracy_score(y_test, svm_rs_predicted))

              precision    recall  f1-score   support

           0       0.65      0.50      0.57       209
           1       0.22      0.23      0.23        35
           2       0.84      0.90      0.87       736

   micro avg       0.79      0.79      0.79       980
   macro avg       0.57      0.54      0.55       980
weighted avg       0.78      0.79      0.78       980

---------------------------------------------------------
The accuracy score of SVM (resampled) is 0.7877551020408163


**confusion matrix**

In [75]:
y_test_re = list(y_test)
for i in range(len(y_test_re)):
    if y_test_re[i] == 0:
        y_test_re[i] = "good"
    if y_test_re[i] == 1:
        y_test_re[i] = "low"
    if y_test_re[i] == 2:
        y_test_re[i] = "medium"
sm_pred_re = list(svm_rs_predicted)
for i in range(len(svm_rs_predicted)):
    if sm_pred_re[i] == 0:
        sm_pred_re[i] = "good"
    if sm_pred_re[i] == 1:
        sm_pred_re[i] = "low"
    if sm_pred_re[i] == 2:
        sm_pred_re[i] = "medium"
svm_rs_confusion = pd.crosstab(pd.Series(y_test_re,name='actual'), pd.Series(sm_pred_re,name='predicted'))

In [76]:
svm_rs_confusion

predicted,good,low,medium
actual,,,
good,104,6,99
low,1,8,26
medium,54,22,660


## Random Forest

In [77]:
pred_rfc = rfc_rs.predict(X_test)
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy is %s" % accuracy_score(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       209
           1       0.00      0.00      0.00        35
           2       0.75      1.00      0.86       736

   micro avg       0.75      0.75      0.75       980
   macro avg       0.25      0.33      0.29       980
weighted avg       0.56      0.75      0.64       980

The RF model accuracy is 0.7510204081632653


/Users/tianqiluke/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [78]:
# relabel back : 0 means good, 1 for low, 2 for medium for better visualization
y_test_re = list(y_test)
for i in range(len(y_test_re)):
  if y_test_re[i] == 0:
    y_test_re[i] = "good"
  if y_test_re[i] == 1:
    y_test_re[i] = "low"
  if y_test_re[i] == 2:
    y_test_re[i] = "medium"
pred_rfc_re = list(pred_rfc)
for i in range(len(pred_rfc_re)):
  if pred_rfc_re[i] == 0:
    pred_rfc_re[i] = "good"
  if pred_rfc_re[i] == 1:
    pred_rfc_re[i] = "low"
  if pred_rfc_re[i] == 2:
    pred_rfc_re[i] = "medium"

In [79]:
y_actu = pd.Series(y_test_re, name='Actual')
y_pred = pd.Series(pred_rfc_re, name='Predicted')
rfc_rs_confusion = pd.crosstab(y_actu, y_pred)

In [80]:
rfc_rs_confusion

Predicted,medium
Actual,
good,209
low,35
medium,736
